# Localization_Classification.ipynb
Date: November 29th, 2018  
Course: ECSE415, McGill University  
Authors:  
*Shawn Vosburg  
Tristan Bouchard  
Alex Masciotra  
Nayem Alam  
Thomas Philippon *

## 3. Localization & Classification

In [25]:
#Import the required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import  random as rand

import csv


In [11]:
#Import images from path 
with open('./MIO-TCD-Localization/MIO-TCD-Localization/gt_train.csv') as f:
    gt_table = csv.reader(f)
    for row in gt_table:
        print(row)